In [1]:

import snscrape.modules.twitter as sntwitter
import os
import datetime
import pandas as pd
df = pd.read_csv('./twibot-22.csv')
df['_id'] = df['id'].map(lambda x: x[1:])


In [3]:

justSome = True
final_df = pd.DataFrame()
for id, label in zip(df['_id'], df['label']):
    user_tweets_list = []

    list1 = []
    id = int(id)

    scrapper = sntwitter.TwitterProfileScraper(id)
    # print(scrapper.is_valid_username())
    try:
        user = scrapper.entity
        list1.append([user.username, user.id, label, user.displayname,
                      user.verified, user.created,
                      user.followersCount, user.friendsCount, user.statusesCount,
                      user.favouritesCount, user.listedCount, user.mediaCount, user.location,
                      user.protected, user.profileImageUrl])

    except:
        print('fail')
        continue

    user_df = pd.DataFrame(list1, columns=['username', 'id', 'label', 'displayname',
                                           'verified', 'created',
                                           'followersCount', 'friendsCount', 'statusesCount',
                                           'favouritesCount', 'listedCount', 'mediaCount', 'location',
                                           'protected', 'profileImageUrl'])
    current_time = datetime.datetime.now()
    for iteration, tweet in enumerate(sntwitter.TwitterUserScraper(id).get_items()):
        if (justSome == True):
            if (iteration > 100):
                break
            # print(tweet.)
        tweetItem = [tweet.date, tweet.user.username, tweet.lang,
                     tweet.hashtags, tweet.replyCount, tweet.retweetCount, tweet.likeCount,
                     tweet.quoteCount, tweet.media, tweet.sourceLabel,  tweet.mentionedUsers]

        user_tweets_list.append(tweetItem)

    user_tweets_df = pd.DataFrame(user_tweets_list, columns=['DateTime',  'Username', 'Language',
                                                             'Hashtags', 'ReplyCount', 'RetweetCount', 'LikeCount',
                                                             'QuoteCount', 'Media', 'Source',  'mentionedUsers'])
    SourceList = user_tweets_df['Source'].unique().tolist()
    userDict = user_df.to_dict('records')[0]
    hashtagList = []
    mentionList = []
    for taglist in user_tweets_df['Hashtags']:
        if (taglist):
            for tag in taglist:
                hashtagList.append(tag)

    for mentions in user_tweets_df['mentionedUsers']:
        if (mentions):
            for mention in mentions:
                mentionList.append(mention.username)

    user_df['totalReplyCount'] = user_tweets_df['ReplyCount'].sum() or 0
    user_df['totalRetweetCount'] = user_tweets_df['RetweetCount'].sum() or 0
    user_df['totalLikeCount'] = user_tweets_df['LikeCount'].sum() or 0
    user_df['totalQuoteCount'] = user_tweets_df['QuoteCount'].sum() or 0
    user_df['totalSource'] = len(
        user_tweets_df['Source'].unique().tolist()) or 0

    user_df.to_csv('BotAccout.csv', mode='a', header=False, index=False)


KeyboardInterrupt: 